In [35]:
import numpy as np

In [ ]:
def gather_object_pointcloud(point_cloud, mask, m_points, device='cuda'):
    def mask_to_indicies(mask, batch_size):
        indices = torch.zeros((batch_size, m_points, 2), dtype=torch.int32)
        for i in range(batch_size):
            pos_indices = np.where(mask[i, :] > 0.5)[0]
            # skip cases when pos_indices is empty
            if len(pos_indices) > 0:
                if len(pos_indices) > m_points:
                    choice = np.random.choice(len(pos_indices),
                                              m_points, replace=False)
                else:
                    choice = np.random.choice(len(pos_indices),
                                              m_points - len(pos_indices), replace=True)
                    choice = np.concatenate((np.arange(len(pos_indices)), choice))
                np.random.shuffle(choice)
                indices[i, :, 1] = torch.from_numpy(pos_indices[choice])
            indices[i, :, 0] = i
        return indices.to(device)

    batch_size = mask.size()[0]
    n_channels = point_cloud.size()[2]

    #indices = torch.autograd.Variable(mask_to_indicies(mask, batch_size))
    indices = mask_to_indicies(mask, batch_size)
    
    object_point_cloud = torch.zeros((mask.size()[0], m_points, n_channels))
    #object_point_cloud = torch.autograd.Variable(object_point_cloud)

    for i in range(batch_size):
        count = 0
        for j in indices[i, :, 1]:
            object_point_cloud[i, count, :] = point_cloud[i, j, :]
            count += 1

    return object_point_cloud.to(device), indices

In [11]:
import torch
import numpy as np
x = np.random.randn(2,3,4)
x

array([[[ 0.13329216, -0.42825173, -0.66352704, -0.25151226],
        [-0.28417719,  0.38093518,  0.48201524, -0.19048918],
        [-2.02072253,  1.30598666, -0.14669808,  1.29994792]],

       [[-0.64609169,  0.97508493,  0.33305814,  1.08070891],
        [ 2.40611802, -0.47110607, -0.45400631,  0.08251184],
        [-0.28001176, -2.27080497, -0.02160056, -0.07199822]]])

In [20]:
import torch
indics = torch.LongTensor(idx)
torch_pc = torch.FloatTensor(pc)
output = torch.gather(torch_pc, 0, indics)
print(output.size())
output

RuntimeError: Expected tensor [2 x 3 x 2], src [2 x 6 x 4] and index [2 x 3 x 2] to have the same size apart from dimension 0 at /opt/conda/conda-bld/pytorch_1535491974311/work/aten/src/TH/generic/THTensorMath.cpp:620

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
idx = np.ones((2,3,2))
x = np.random.randn(2,3,4)

In [5]:
x

array([[[-1.56831218, -0.55415761,  0.51438201,  1.35513448],
        [ 1.56367027, -0.73864353, -0.71441799,  0.78760628],
        [ 0.08812818, -0.36317287,  0.60575778,  1.3710668 ]],

       [[ 0.33651575,  0.66759067,  0.98694863, -1.39022862],
        [-0.44095884, -1.5241819 , -0.67308512, -0.15392798],
        [ 0.72538415, -0.739054  , -0.34964515,  1.31625855]]])

In [14]:
indics_tf = tf.constant(idx, dtype=np.int32)

pc_tf = tf.convert_to_tensor(pc)

output_tf = tf.gather_nd(pc_tf, indics_tf)
output_tf

<tf.Tensor 'GatherNd_1:0' shape=(2, 3, 4) dtype=float64>

In [36]:
sess = tf.InteractiveSession()
output_tf_eval = output_tf.eval()
print(output_tf_eval.shape)
sess.close()

(2, 3, 4)


In [16]:
o = gather_x(pc, idx)
print(o.shape)
print(o)

(2, 3, 4)
[[[-0.2224 -0.5145  2.2887  0.    ]
  [-0.1772  0.1358 -3.0995  0.    ]
  [ 0.7007  0.5813  2.545   0.    ]]

 [[ 0.      0.      0.      0.    ]
  [ 0.      0.      0.      0.    ]
  [ 0.      0.      0.      0.    ]]]


In [12]:
point_cloud = x
indices = np.ones((2,3,2), dtype=np.int32)
batch_size = 2


def gather_x(point_cloud, indices):
    object_point_cloud = np.zeros((batch_size, 3, 4))
    for i in range(batch_size):
        count = 0
        for j in indices[i, :, 1]:
            object_point_cloud[i, count, :] = point_cloud[i, j, :]
            count += 1
    return object_point_cloud

In [33]:
idx = np.asarray([[[0, 1],[0, 2],[0, 4]], [[1, 2],[1, 2], [1, 2]]])
idx_torch = np.asarray([[[1, 1,1,1],[2,2,2, 2],[4,4,4, 4]], [[2,2,2, 2],[2,2,2, 2], [2,2,2,2]]])

In [34]:
pc = np.asarray([[[ 0.3298, -0.3560,  1.5519,  0.0000],
         [-0.2224, -0.5145,  2.2887,  0.0000],
         [-0.1772,  0.1358, -3.0995,  0.0000],
         [-0.9271, -0.4670, -3.5719,  0.0000],
         [ 0.7007,  0.5813,  2.5450,  0.0000],
         [ 0.2962,  0.6205,  0.2858,  0.1900]],

        [[ 0.4187, -0.2572,  1.7973,  0.0000],
         [ 0.1225,  1.0131,  0.6915,  0.2000],
         [ 0.0000,  0.0000,  0.0000,  0.0000],
         [-0.4852,  0.9913,  0.6028,  0.2900],
         [-1.5547, -0.0345, 18.2407,  0.3100],
         [ 0.4657,  0.6077, -0.4934,  0.0000]]])

In [13]:
o = gather_x(pc, idx)
print(o.shape)
print(o)

(2, 3, 4)
[[[-0.2224 -0.5145  2.2887  0.    ]
  [-0.1772  0.1358 -3.0995  0.    ]
  [ 0.7007  0.5813  2.545   0.    ]]

 [[ 0.      0.      0.      0.    ]
  [ 0.      0.      0.      0.    ]
  [ 0.      0.      0.      0.    ]]]


In [31]:
idx_test = np.asarray([[[0,0,0,0],
  [1,1,1,1],
  [2,2,2,2]],
 [[3,3,3,3],
  [4,4,4,4],
  [5,5,5,5]]])
print(idx_test.shape)
torch.gather(torch_pc, 1, torch.LongTensor(idx_test))

(2, 3, 4)


tensor([[[ 0.3298, -0.3560,  1.5519,  0.0000],
         [-0.2224, -0.5145,  2.2887,  0.0000],
         [-0.1772,  0.1358, -3.0995,  0.0000]],

        [[-0.4852,  0.9913,  0.6028,  0.2900],
         [-1.5547, -0.0345, 18.2407,  0.3100],
         [ 0.4657,  0.6077, -0.4934,  0.0000]]])

In [32]:
torch_pc

tensor([[[ 0.3298, -0.3560,  1.5519,  0.0000],
         [-0.2224, -0.5145,  2.2887,  0.0000],
         [-0.1772,  0.1358, -3.0995,  0.0000],
         [-0.9271, -0.4670, -3.5719,  0.0000],
         [ 0.7007,  0.5813,  2.5450,  0.0000],
         [ 0.2962,  0.6205,  0.2858,  0.1900]],

        [[ 0.4187, -0.2572,  1.7973,  0.0000],
         [ 0.1225,  1.0131,  0.6915,  0.2000],
         [ 0.0000,  0.0000,  0.0000,  0.0000],
         [-0.4852,  0.9913,  0.6028,  0.2900],
         [-1.5547, -0.0345, 18.2407,  0.3100],
         [ 0.4657,  0.6077, -0.4934,  0.0000]]])

In [37]:
output_tf_eval

array([[[-0.2224, -0.5145,  2.2887,  0.    ],
        [-0.1772,  0.1358, -3.0995,  0.    ],
        [ 0.7007,  0.5813,  2.545 ,  0.    ]],

       [[ 0.    ,  0.    ,  0.    ,  0.    ],
        [ 0.    ,  0.    ,  0.    ,  0.    ],
        [ 0.    ,  0.    ,  0.    ,  0.    ]]])

In [39]:
torch_pc = torch.FloatTensor(pc)
torch_idx_tensor = torch.LongTensor(idx_torch)

output_torch = torch.gather(torch_pc, 1, torch_idx_tensor)

In [41]:
output_torch

tensor([[[-0.2224, -0.5145,  2.2887,  0.0000],
         [-0.1772,  0.1358, -3.0995,  0.0000],
         [ 0.7007,  0.5813,  2.5450,  0.0000]],

        [[ 0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000]]])

In [42]:
output_tf_eval

array([[[-0.2224, -0.5145,  2.2887,  0.    ],
        [-0.1772,  0.1358, -3.0995,  0.    ],
        [ 0.7007,  0.5813,  2.545 ,  0.    ]],

       [[ 0.    ,  0.    ,  0.    ,  0.    ],
        [ 0.    ,  0.    ,  0.    ,  0.    ],
        [ 0.    ,  0.    ,  0.    ,  0.    ]]])

In [52]:
torch.from_numpy(np.random.randn((6))).unsqueeze(1).repeat(1,4)

tensor([[-1.3644, -1.3644, -1.3644, -1.3644],
        [-0.1275, -0.1275, -0.1275, -0.1275],
        [-2.3967, -2.3967, -2.3967, -2.3967],
        [-0.4475, -0.4475, -0.4475, -0.4475],
        [-0.6955, -0.6955, -0.6955, -0.6955],
        [ 0.4388,  0.4388,  0.4388,  0.4388]], dtype=torch.float64)

In [51]:
_

torch.Size([6, 4])